In [2]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import time

In [3]:

# define cnn model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [7]:
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    pyplot.savefig('vgg16_mlp_layers_plot.png')
    pyplot.close()

In [8]:
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]
    # prepare iterator
    train_it = datagen.flow_from_directory('images/train/',
    class_mode='binary', batch_size=64, target_size=(224, 224))
    test_it = datagen.flow_from_directory('images/test/',
    class_mode='binary', batch_size=64, target_size=(224, 224))

    start_time = time.time()

    history = model.fit(train_it, steps_per_epoch=len(train_it),
    validation_data=test_it, validation_steps=len(test_it), epochs=7, verbose=1)

    end_time = time.time()
    # evaluate model
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
   
    training_time = end_time - start_time
    print("Training Time: %.2f seconds" % training_time)

    print("Training Loss:")
    print(history.history['loss'])
    print("Training Accuracy:")
    print(history.history['accuracy'])
    print("Testing Accuracy:")
    print('> %.3f' % (acc * 100.0))
    model.summary()
   
    # learning curves
    summarize_diagnostics(history)

In [9]:
run_test_harness()

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/7
3/3 [==============================] - 30s 10s/step - loss: 33.0842 - accuracy: 0.4500 - val_loss: 0.6820 - val_accuracy: 0.8250
Epoch 2/7
3/3 [==============================] - 28s 11s/step - loss: 1.5730 - accuracy: 0.7250 - val_loss: 0.0503 - val_accuracy: 0.9750
Epoch 3/7
3/3 [==============================] - 29s 9s/step - loss: 0.1086 - accuracy: 0.9625 - val_loss: 0.2137 - val_accuracy: 0.9000
Epoch 4/7
3/3 [==============================] - 29s 9s/step - loss: 0.1284 - accuracy: 0.9438 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 5/7
3/3 [==============================] - 29s 9s/step - loss: 0.0234 - accuracy: 0.9875 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 6/7
3/3 [==============================] - 30s 10s/step - loss: 0.0324 - accuracy: 0.9875 - val_loss: 0.0082 - val_accuracy: 1.0000
Epoch 7/7
3/3 [==============================] - 29s 10s/step - loss: 0.0301 - accuracy: 0.98